In [125]:
import plotly.graph_objects as go
from dataTable import *
from weaponData import *
from dash import *

# 获取各种设置
settings = et.parse("settings.xml").getroot()
generalSettings = settings.find("general")
graphSettings = settings.find("graphSettings")
health = int(generalSettings.get("health"))
interpolation = generalSettings.get("interpolation")
xAxis = graphSettings.get("xAxis")
yAxis = graphSettings.get("yAxis")
def getTitleByWeapon(weapon):
    return weapon.name + " (ID: {})".format(str(weapon.id))
def traceWeapon(weaponToTrace, hide = False):
    hoverFields = graphSettings.get("hoverFields").split(',')
    global fig
    id = 0
    try:
        id = int(weaponToTrace)
    except:
        # 把weaponToPlot从武器数据转换到ID
        id = getIDByName(weaponToTrace)
    weapon = WeaponData(id)
    killData = weapon.getKillData(health)
    x=killData[xAxis].tolist()
    y=killData[yAxis].tolist()
    # 获取悬浮菜单中的数据
    hoverText = []
    # 遍历X轴的所有数据
    for i in x:
        text = ''
        # 找到悬浮菜单中对应当前X的数值
        for j in hoverFields:
            text += j + ': ' + str(killData[killData[xAxis] == i][j].iloc[0]) + '<br>'
        hoverText.append(text)
    trace = go.Scatter(x=x, y=y, line_shape = interpolation, name = getTitleByWeapon(weapon),
                             hovertemplate = 
                                 '<br><b>' + yAxis + ': %{y}</b><br>' + 
                                 '<b>' + xAxis + ': %{x}</b><br>' +
                                 '%{text}', 
                                 text = hoverText,
                                 visible=not(hide)
                             )
    return trace
def plotWeapon(weaponToPlot, hide = False):
    fig.add_trace(traceWeapon(weaponToPlot, hide = hide))
def traceAll(hide = False):
    output = []
    for i in weapons:
        output.append(traceWeapon(i.id, hide=hide))
    return output
def plotAll(figure, hide = False):
    figure.add_traces(traceAll(hide = hide))
def updateLayout(figure):
    figure.update_layout(
    hovermode='x unified', 
    xaxis_title=xAxis, yaxis_title=yAxis, 
    width=float(generalSettings.get("sizeX")), height=float(generalSettings.get("sizeY")),
    )
def getMasterFigure(hide = False):
    fig = go.Figure()
    plotAll(fig, hide = hide)
    updateLayout(fig)
    return fig

# 注意事项：
如果你对任何此文件之外的文件进行了修改（代码、数据、settings.xml设置文件等），记得在运行之前点Restart重启kernel，否则数据和代码都不会更新。
关于该程序的具体设置，请见README.MD
# IMPORTANT:
If you have modified any files outside of this file (code, data, settings.xml settings file, etc.), remember to restart the kernel by clicking on the Restart button before running, otherwise the data and code will not be updated.
For specific settings of this program, please refer to README-EN.MD

In [ ]:
app = Dash()

weaponNames = [i.name for i in weapons]
# App layout
app.layout = [
    dcc.Dropdown(weaponNames, 'Select Weapon', id='weaponsDropdown'),
    html.Button('Toggle', id='toggleVisibility'),
    html.Button('Show All', id='showAll'),
    html.Button('Clear All', id='clearAll'),
    dcc.Graph(id="graph", figure=getMasterFigure(hide=True)),
]


@callback(
    Output('graph', 'figure'),
    Input('toggleVisibility', 'n_clicks'), 
    Input('weaponsDropdown', 'value'),
    Input('showAll', 'n_clicks'), 
    Input('clearAll', 'n_clicks'), 
    Input('graph', 'figure'),
    prevent_initial_call=True
)
def OnInteract(toggle, dropDown, b1, b2, currentFig):
    id = ctx.triggered_id
    updatedFigure = go.Figure(currentFig)
    #print(updatedFigure)
    if id == 'toggleVisibility':
        print(dropDown)
    # TODO 下拉菜单组件
        if (dropDown != None): 
            weapon = getWeaponByName(dropDown)
            for i in range(len(updatedFigure.data)):
                #print(updatedFigure.data[i].name)
                if (updatedFigure.data[i].name == getTitleByWeapon(weapon)):
                    updatedFigure.data[i].visible = not(updatedFigure.data[i].visible)
                    print(updatedFigure.data[i]) 
            return updatedFigure
    # TODO 显示所有按钮组件
    elif id == 'showAll':
        return getMasterFigure(hide=False)
    # TODO 清除所有按钮组件
    elif id == 'clearAll':
        return getMasterFigure(hide=True)
    updateLayout(updatedFigure)
    return updatedFigure
# Run the app
if __name__ == '__main__':
    app.run(debug=False)

[2025-04-23 21:57:55,167] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\zizhu\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\zizhu\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\zizhu\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\zizhu\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\zizhu\AppData\Local\Programs\Python\Python310\lib\site-packages\dash\dash.py", line 1413, in dispatch
    ctx.run(
  File "c:\Users\z

1
